# Various Functions#

In [2]:
import numpy as np
from scipy.stats import unitary_group
import time

## Matrix generation

Takes an array of eigenvalues and creates a random matrix with said eigenvalues via similarity-transformation.

In [5]:
def gen_rand_mat(eig, compl = False):
    n = eig.shape[0]
    if compl:
        B = np.random.rand(n,n) + np.random.rand(n,n)*compl*1j
    else:
        B = np.random.rand(n,n)
    return np.linalg.inv(B)@np.diag(eig)@B

Generates a random Hessenberg-Matrix. In the complex case it is a tridiagonal matrix.

In [6]:
def gen_hess(n, compl = False):
    if compl:
        a = np.random.rand(n) + np.random.rand(n)*1j
        b = np.random.rand(n-1) + np.random.rand(n-1)*1j
        c = np.random.rand(n-1) + np.random.rand(n-1)*1j
        return np.diag(b, -1) + np.diag(a,0) + np.diag(c, 1)
    else:
        A = np.zeros((n,n))
        a = np.random.rand(n)*50
        for j in range(n-1):
            b = np.random.rand(n-j-1)*50
            A = A + np.diag(b,j+1)
        return np.diag(np.random.rand(n-1)*50, -1) + A

Takes an array of eigenvalues and creates a hermitean matrix with said eigenvalues.

In [7]:
def gen_hermite(eig, compl = False):
    n = eig.shape[0]
    B = unitary_group.rvs(n)
    return B.T.conj()@np.diag(eig)@B

## Errors and Runtime

Takes two arrays, sorts them, shortens the presumably longer one and returns the norm of the difference (inf is for the max-norm).

In [8]:
def error(eig_1, eig_2):
    eig_1 = np.sort(eig_1)
    eig_2 = np.sort(eig_2)
    eig_2 = eig_2[:len(eig_1)]
    return np.linalg.norm(eig_1 - eig_2)

Takes two array, returns the sum of the norm-difference in the first k and last k entries.

In [9]:
def selective_error(eig_1, eig_2, k = 1):
    n_1 = len(eig_1)
    n_2 = len(eig_2)
    error_1 = np.linalg.norm(eig_1[:k] - eig_2[:k])
    error_2 = np.linalg.norm(eig_1[:n_1-k-1:-1] - eig_2[:n_2-k-1:-1])
    return error_1 + error_2

Returns the time needed for one function evaluation of f given input A.

In [10]:
def runtime(f, A):
    start = time.time()
    S = f(A)
    end = time.time()
    return end - start

Returns the error of some f(A) to some eigv as well as the count as returned from f, the case $l > 0$ is implemented for Lanczos, the casse $l > 0$ for Arnoldi.

In [11]:
def runtime_error(f, A, eigv, k = 0, l = 0):
    if k == 0:
        l, S, count = f(A)
        S = error(S, eigv)
    elif l == 0:
        l, S, count = f(A, k)
        S = error(S, eigv)
    else:
        l, S, count = f(lambda x: A@x, l, k)
        S = error(S, eigv)
    return count, S